In [1]:
from AOC_utils import get_day
import numpy as np

input_data = get_day(day=10, year=2025)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 10 input already downloaded
157


['[##...#] (1,3,4,5) (2,3,5) (0,2,3) (0,2,3,4,5) (1,2,4) (0,1,2,3) {24,27,40,30,25,17}',
 '[#....] (0,2) (0,1,4) (0) (0,4) (0,3,4) (0,1,2,3) {53,16,16,24,27}',
 '[#####.###] (4,6) (0,5,6,8) (0,1,3,5,6,8) (0,1,2,3,4,5,7,8) (2,3) (1,2,3,4,6,7) (0,2,5,6,8) (2,3,4,5) (0,1,2,3,5,6,8) {168,164,176,171,51,173,194,30,168}',
 '[..##] (1,3) (0,2,3) (0,1) (2,3) {30,15,29,34}',
 '[....#.#.] (0,1,5) (2,4,6) (2,3) (2,3,7) (0,1,4,5,6,7) (0,2,3,4,5,6) {25,14,42,28,32,25,32,13}',
 '[.###] (0,1,2,3) (1,2,3) {0,9,9,9}',
 '[####....#] (2) (3,4,5) (0,1,2,3,8) (0,1,3,4,6,7,8) (1,2,6) (0,4,5,6,7,8) (6,7) (0,5,6,7) (0,1,2,3,6,7) (0,2,4,5,6,7) {31,36,158,26,28,25,52,32,18}']

In [2]:
example = '''
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
'''.split('\n')[1:-1]

In [3]:
from z3.z3 import Solver, sat, Reals, Int, Optimize

In [4]:
def solve_day(data):
    part1 = 0; part2 = 0

    all_lights = [np.array(list(x.split(' ')[0][1:-1].replace('#', '0').replace('.', '1')), dtype=int).astype(bool) for x in data]
    all_buttons = [[np.array(y[1:-1].split(','), dtype=int) for y in x.split(' ')[1:-1]] for x in data]
    all_joltages = [np.array(x.split(' ')[-1][1:-1].split(','), dtype=int) for x in data]

    from collections import deque
    def bfs(lights, buttons):

        initial = tuple(lights)
        queue = deque([(initial, [])])

        while queue:
            state, buttons_used = queue.popleft()
            state_array = np.array(state, dtype=bool)

            for b in range(len(buttons)):
                if b in buttons_used:
                    continue

                button = buttons[b]
                new_state = state_array.copy()
                new_state[button] = ~new_state[button]

                if new_state.all():
                    return len(buttons_used) + 1
                
                queue.append((tuple(new_state), buttons_used + [b]))

        raise ValueError("No solution found")
    
    for lights, buttons in zip(all_lights, all_buttons):
        min_presses = bfs(lights, buttons)
        part1 += min_presses


    for buttons, joltages in zip(all_buttons, all_joltages):
        lines = []
        for button in buttons:
            new_line = np.zeros(len(joltages), dtype=int)
            new_line[button] = 1
            lines.append(new_line)
            
        multipliers = [Int('x%s' % i) for i in range(len(buttons))]
        targets = [Int('t%s' % i) for i in range(len(joltages))]

        s = Optimize()

        for i in range(len(joltages)):
            equation = sum(lines[j][i]*multipliers[j] for j in range(len(buttons)))
            s.add(equation == targets[i])
            s.add(targets[i] == joltages[i])
        
        for i in range(len(buttons)):
            s.add(multipliers[i] >= 0)

        s.minimize(sum(multipliers))

        if s.check() == sat:
            m = s.model()
            presses = [m[multipliers[i]].as_long() for i in range(len(multipliers))]
            total_presses = sum(presses)
            part2 += total_presses


    
    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example)

part 1: 7
part 2: 33


In [5]:
solve_day(input_data)

part 1: 385
part 2: 16757
